#   LangChain Tool 활용

---

## 환경 설정 및 준비

`(1) Env 환경변수`

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) 기본 라이브러리`

In [2]:
import os
from glob import glob

from pprint import pprint
import json

`(3) Langsmith tracing 설정`

In [3]:
# Langsmith tracing 여부를 확인 (true: langsmith 추척 활성화, false: langsmith 추척 비활성화)
import os
print(os.getenv('LANGSMITH_TRACING'))

true


---

## **내장 도구 (Built-in Tool)**

- **내장 도구**는 시스템에 **기본적으로 포함**된 사전 정의된 도구들의 집합

- 파일 처리, 웹 검색, 코드 실행 등 **자주 사용되는 기본 기능**들을 즉시 활용 가능

- 별도의 구현 없이 **바로 사용**할 수 있어 개발 시간과 노력을 절약 가능

---

### **웹 검색(Tavily Search API)**

- **Tavily**는 AI 에이전트(LLM)를 위해 특별히 설계된 검색 엔진

- 검색 결과로 **제목, URL, 컨텐츠, 답변** 등 상세 정보 제공

- 개발자는 **월 1,000회** 무료 검색 쿼터 사용 가능

- **설치**: `pip install tavily-python` 또는 `uv add tavily-python`

- **인증키**: `TAVILY_API_KEY`

`(1) 도구 정의`

In [4]:
from langchain_community.tools import TavilySearchResults

search_web = TavilySearchResults(
    max_results=5,  # 반환할 결과의 수
    search_depth="advanced",  # 검색 깊이: basic 또는 advanced
    include_answer=True,  # 결과에 직접적인 답변 포함
    include_raw_content=True,  # 페이지의 원시 콘텐츠 포함
    include_images=True,  # 결과에 이미지 포함
    # include_domains=[...],  # 특정 도메인으로 검색 제한
    # exclude_domains=[...],  # 특정 도메인 제외
    # name="...",  # 기본 도구 이름 덮어쓰기
    # description="...",  # 기본 도구 설명 덮어쓰기
    # args_schema=...,  # 기본 args_schema 덮어쓰기
)

C:\Users\User\AppData\Local\Temp\ipykernel_12784\1142060768.py:3: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  search_web = TavilySearchResults(


`(2) 도구 직접 호출`
- 자연어 쿼리를 도구에 전달

In [5]:
result = search_web.invoke("한국 시장에서 거래되는 ETF 종목은 모두 몇 개인가요?")
pprint(result)

[{'content': '16일 한국거래소에 따르면 국내 ETF 시장 순자산 규모는 158조3001억원으로 집계됐다. 870개 종목이 '
             "상장돼있다. 2002년 10월 14일 'KODEX 200' 등 4",
  'score': 0.8868231,
  'title': '국내 ETF 상품만 870개, 전세계 4위…앞서나가는 한국 ETF 시장',
  'url': 'https://www.hankyung.com/article/202407142443i'},
 {'content': '국내 코스닥 시장에 상장되어 있는 종목 중 시가총액, 유동성, 업종분포 등을 고려하여 선정된 150개 종목으로 '
             '구성된 ETF입니다. 한국의 제 2의 주식시장, 코스닥(KOSDAQ).',
  'score': 0.8437023,
  'title': 'Kodex 코스닥 150 - 삼성자산운용',
  'url': 'https://www.samsungfund.com/etf/product/view.do?id=2ETF54'},
 {'content': '2017년 8월말 기준 국내 상장지수펀드(Exchange Trade Funds: ETF)의 순자산규모 및 상장종목. '
             '□. 수는 각각 28.6조원과 302개를 기록하였고 2002년',
  'score': 0.7557276,
  'title': '국내 ETF 성장 현황 및 특징 - 자본시장연구원',
  'url': 'https://www.kcmi.re.kr/common/downloadw.php?fid=21648&fgu=002001&fty=004003'},
 {'content': '금리인상 등으로 시장 변동성이 확대되는 중에도 상장종목수. 622 ... □ (글로벌 위상) 한국 ETF시장은 '
             '상장종목수 6위, 순자산총액. 12위, 일',
  'score': 0.74161834,
  'title': '[PDF] 보 도 자 료 - 한국금융연구원',
  'url

`(3) ToolCall을 사용한 호출`
- 모델에서 생성된 `ToolCall`을 사용하여 도구를 호출

In [6]:
from langchain_openai import ChatOpenAI

# 모델 생성
model = ChatOpenAI(model="gpt-4.1-mini")

# 모델에 도구 등록 
model_with_tools = model.bind_tools([search_web])

# 사용자 쿼리를 입력하여 ToolCall 생성
response = model_with_tools.invoke("한국 시장에서 거래되는 ETF 종목은 모두 몇 개인가요?")

# ToolCall 출력 
pprint(response.tool_calls)

[{'args': {'query': '한국 시장 ETF 종목 수'},
  'id': 'call_KBxAwSRUDX8ro3svpFWAYlid',
  'name': 'tavily_search_results_json',
  'type': 'tool_call'}]


In [7]:
# ToolCall을 사용하여 도구 실행
model_generated_tool_call = response.tool_calls[0]
tool_msg = search_web.invoke(model_generated_tool_call)

# 도구 실행 결과 출력
pprint(tool_msg.content)

('[{"title": "Kodex 코스닥 150 - 삼성자산운용", "url": '
 '"https://www.samsungfund.com/etf/product/view.do?id=2ETF54", "content": "국내 '
 '코스닥 시장에 상장되어 있는 종목 중 시가총액, 유동성, 업종분포 등을 고려하여 선정된 150개 종목으로 구성된 ETF입니다. 한국의 제 '
 '2의 주식시장, 코스닥(KOSDAQ).", "score": 0.8342047}, {"title": "[PDF] [Global ETF '
 'Insight] Passive ETF: 불가피한 경쟁적 시장, 종목 선택의 ...", "url": '
 '"https://www.hanaw.com/main/research/research/download.cmd?bbsSeq=1281802&attachFileSeq=1&bbsId=&dbType=&bbsCd=1260", '
 '"content": "미국 ETF 시장 규모는 2020년 연말의 545.4억달러에서 1,070.8억달러까지, 한국 ETF 시장은 '
 '2020년 52.0조원에 서 2025년 1월말 현재 190.1조원 수준까지", "score": 0.69249696}, {"title": '
 '"국내 ETF 현황 - Fund&ETFs - FnIndex", "url": "https://fnindex.co.kr/fund/etf", '
 '"content": "한국투자밸류자산운용. ETF. 2. VITA MZ소비액티브 · VITA 밸류알파액티브. 타임폴리오 ... 국내 '
 'ETF 현황. 국내 ETF 현황 자료입니다. 상장종목수; 순자산; 거래량; 수익률.", "score": 0.6578248}, '
 '{"title": "ETF종목발행현황 - KSD 증권정보포털", "url": '
 '"https://m.seibro.or.kr/cnts/etf/selectPublishInfo.do", "content": "| FOCUS '
 'KRX300;) | 브이아이

In [8]:
# `artifact` 속성 출력
tool_msg.artifact

{'query': '한국 시장 ETF 종목 수',
 'follow_up_questions': None,
 'answer': 'As of 2025, there are over 240 ETF products listed in the South Korean market. The largest ETF by assets is KODEX 200. The market size of the Korean ETF market has grown significantly since 2020.',
 'images': ['https://newsimg.sedaily.com/2022/11/07/26DJ92T9IN_1.png',
  'https://cdnimage.dailian.co.kr/news/202209/news_1663911682_1155566_m_2.jpeg',
  'https://cfnimage.commutil.kr/phpwas/restmb_allidxmake.php?pp=002&idx=999&simg=20230421185017046379249a1ae6311823515204.jpg&nmt=18',
  'https://img.hankyung.com/photo/202108/01.26774180.1.jpg',
  'https://thumbnews.nateimg.co.kr/view610/news.nateimg.co.kr/orgImg/fn/2023/05/24/202305241646513317_l.png'],
 'results': [{'url': 'https://www.samsungfund.com/etf/product/view.do?id=2ETF54',
   'title': 'Kodex 코스닥 150 - 삼성자산운용',
   'content': '국내 코스닥 시장에 상장되어 있는 종목 중 시가총액, 유동성, 업종분포 등을 고려하여 선정된 150개 종목으로 구성된 ETF입니다. 한국의 제 2의 주식시장, 코스닥(KOSDAQ).',
   'score': 0.8342047,
   'raw_con

In [9]:
pprint(tool_msg.artifact)

{'answer': 'As of 2025, there are over 240 ETF products listed in the South '
           'Korean market. The largest ETF by assets is KODEX 200. The market '
           'size of the Korean ETF market has grown significantly since 2020.',
 'follow_up_questions': None,
 'images': ['https://newsimg.sedaily.com/2022/11/07/26DJ92T9IN_1.png',
            'https://cdnimage.dailian.co.kr/news/202209/news_1663911682_1155566_m_2.jpeg',
            'https://cfnimage.commutil.kr/phpwas/restmb_allidxmake.php?pp=002&idx=999&simg=20230421185017046379249a1ae6311823515204.jpg&nmt=18',
            'https://img.hankyung.com/photo/202108/01.26774180.1.jpg',
            'https://thumbnews.nateimg.co.kr/view610/news.nateimg.co.kr/orgImg/fn/2023/05/24/202305241646513317_l.png'],
 'query': '한국 시장 ETF 종목 수',
 'response_time': 0.93,
 'results': [{'content': '국내 코스닥 시장에 상장되어 있는 종목 중 시가총액, 유동성, 업종분포 등을 고려하여 선정된 '
                         '150개 종목으로 구성된 ETF입니다. 한국의 제 2의 주식시장, 코스닥(KOSDAQ).',
              'raw_cont

`(4) LLM 체인과 연계`
- Tavily 도구를 언어 모델에 바인딩하고 사용자 입력을 처리하는 체인을 생성

In [10]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain
from langchain_core.messages import ToolMessage

import datetime

# 언어 모델 초기화
llm = ChatOpenAI(model="gpt-4.1-mini")

# 프롬프트 템플릿 정의
today = datetime.datetime.today().strftime("%Y-%m-%d")
prompt = ChatPromptTemplate(
    [
        ("system", f"당신은 도움이 되는 어시스턴트입니다. 오늘 날짜는 {today}입니다."),
        ("human", "{user_input}"),
        ("placeholder", "{messages}"),
    ]
)

# Tavily 도구를 모델에 바인딩
llm_with_tools = llm.bind_tools([search_web])

# 체인 생성
llm_chain = prompt | llm_with_tools

@chain  # 함수를 체인으로 사용하기 위해 데코레이터 추가
def tool_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    
    # 도구 호출 결과를 처리
    tool_msgs = []
    artifacts = []  # artifact를 저장할 리스트
    for tool_call in ai_msg.tool_calls:
        tool_result = search_web.invoke(tool_call)
        tool_msgs.append(ToolMessage(content=str(tool_result.content), tool_call_id=tool_call["id"]))
        artifacts.append(tool_result.artifact)  # artifact 저장
    
    # 최종 결과 반환
    final_response = llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)
    
    return final_response, artifacts

# 체인 호출
response, artifacts = tool_chain.invoke("한국 시장에서 거래되는 ETF 종목은 모두 몇 개인가요?")
print("===== 최종 응답 =====")
print(response.content)

===== 최종 응답 =====
2025년 6월 기준으로 한국 시장에서 거래되는 ETF 종목은 약 360여 개 정도로 확인됩니다. 

더 상세한 종목별 리스트나 최신 정보가 필요하시다면, 한국 거래소나 자산운용사 공식 홈페이지에서 확인할 수 있습니다.


In [11]:
# artifact를 별도로 출력
print("===== Artifact 정보 =====")
for artifact in artifacts:
    print(json.dumps(artifact, indent=2, ensure_ascii=False))

===== Artifact 정보 =====
{
  "query": "한국 시장 ETF 종목 수",
  "follow_up_questions": null,
  "answer": "As of 2025, there are over 240 ETF products listed in the South Korean market. The largest ETF by assets is KODEX 200. The market size of the Korean ETF market has grown significantly since 2020.",
  "images": [
    "https://newsimg.sedaily.com/2022/11/07/26DJ92T9IN_1.png",
    "https://cdnimage.dailian.co.kr/news/202209/news_1663911682_1155566_m_2.jpeg",
    "https://cfnimage.commutil.kr/phpwas/restmb_allidxmake.php?pp=002&idx=999&simg=20230421185017046379249a1ae6311823515204.jpg&nmt=18",
    "https://img.hankyung.com/photo/202108/01.26774180.1.jpg",
    "https://thumbnews.nateimg.co.kr/view610/news.nateimg.co.kr/orgImg/fn/2023/05/24/202305241646513317_l.png"
  ],
  "results": [
    {
      "url": "https://www.samsungfund.com/etf/product/view.do?id=2ETF54",
      "title": "Kodex 코스닥 150 - 삼성자산운용",
      "content": "국내 코스닥 시장에 상장되어 있는 종목 중 시가총액, 유동성, 업종분포 등을 고려하여 선정된 150개 종목으로 구성된 ETF입니다.

---

##  **사용자 정의 도구 (Custom Tool)**


- **사용자 정의 도구**는 개발자가 직접 설계하고 구현하는 **맞춤형 함수나 도구**를 의미

- LLM이 호출할 수 있는 **고유한 기능**을 정의하여 특정 작업에 최적화된 도구 생성 가능

- 개발자는 도구의 **입력값, 출력값, 기능**을 자유롭게 정의하여 유연한 확장성 확보

---

### 1. **외부 API 연동** 

- LangChain에서는 **사용자 정의 도구**를 통해 외부 API와의 연동이 가능

- **Tool** 클래스를 상속받아 필요한 기능을 구현하며, `name`과 `description`을 필수로 정의

- 도구는 단일 기능을 수행하는 **함수 형태**로 구현되며, 입력과 출력이 명확하게 설정함

`(1) Yahoo Finance API` 

In [12]:
#  yfinance 설치 : pip install yfinance 또는 uv add yfinance
import yfinance as yf

dat = yf.Ticker("MSFT")

In [13]:
# 기업정보
dat.info

{'address1': 'One Microsoft Way',
 'city': 'Redmond',
 'state': 'WA',
 'zip': '98052-6399',
 'country': 'United States',
 'phone': '425 882 8080',
 'website': 'https://www.microsoft.com',
 'industry': 'Software - Infrastructure',
 'industryKey': 'software-infrastructure',
 'industryDisp': 'Software - Infrastructure',
 'sector': 'Technology',
 'sectorKey': 'technology',
 'sectorDisp': 'Technology',
 'longBusinessSummary': 'Microsoft Corporation develops and supports software, services, devices and solutions worldwide. The Productivity and Business Processes segment offers office, exchange, SharePoint, Microsoft Teams, office 365 Security and Compliance, Microsoft viva, and Microsoft 365 copilot; and office consumer services, such as Microsoft 365 consumer subscriptions, Office licensed on-premises, and other office services. This segment also provides LinkedIn; and dynamics business solutions, including Dynamics 365, a set of intelligent, cloud-based applications across ERP, CRM, power 

In [14]:
# 주요 일정
dat.calendar

{'Dividend Date': datetime.date(2025, 9, 11),
 'Ex-Dividend Date': datetime.date(2025, 8, 21),
 'Earnings Date': [datetime.date(2025, 7, 29), datetime.date(2025, 8, 2)],
 'Earnings High': 3.57,
 'Earnings Low': 3.29784,
 'Earnings Average': 3.37462,
 'Revenue High': 74464996940,
 'Revenue Low': 72570000000,
 'Revenue Average': 73792496670}

In [15]:
# 2022년 1월 3일 ~ 4일의 데이터를 판다스 데이터프레임으로 출력 
result = dat.history(start="2022-01-03", end="2022-01-05") 
result

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2022-01-03 00:00:00-05:00,325.620654,328.193765,320.212247,325.038055,28865100,0.0,0.0
2022-01-04 00:00:00-05:00,325.115693,325.474983,316.658401,319.464569,32674300,0.0,0.0


In [16]:
# 인덱스 초기화 
result = result.reset_index()
result

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2022-01-03 00:00:00-05:00,325.620654,328.193765,320.212247,325.038055,28865100,0.0,0.0
1,2022-01-04 00:00:00-05:00,325.115693,325.474983,316.658401,319.464569,32674300,0.0,0.0


In [17]:
# 날짜 부분만 추출
result['Date'] = result['Date'].dt.strftime('%Y-%m-%d')

result

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2022-01-03,325.620654,328.193765,320.212247,325.038055,28865100,0.0,0.0
1,2022-01-04,325.115693,325.474983,316.658401,319.464569,32674300,0.0,0.0


In [18]:
# 데이터프레임을 딕셔너리로 변환
result_dict = result.to_dict(orient='records') 
result_dict

[{'Date': '2022-01-03',
  'Open': 325.62065382806765,
  'High': 328.1937646958434,
  'Low': 320.2122465111482,
  'Close': 325.0380554199219,
  'Volume': 28865100,
  'Dividends': 0.0,
  'Stock Splits': 0.0},
 {'Date': '2022-01-04',
  'Open': 325.1156931532872,
  'High': 325.47498337685073,
  'Low': 316.6584013250783,
  'Close': 319.4645690917969,
  'Volume': 32674300,
  'Dividends': 0.0,
  'Stock Splits': 0.0}]

`(2) 데이터 연동 및 출력 포맷` 

In [ ]:
# 외부 API 연동하는 함수 (yfinance 사용)

from langchain_core.tools import ToolException
from typing import Dict, Optional
from datetime import datetime, timedelta
import yfinance as yf

def get_stock_price(symbol: str, date: Optional[str] = None) -> Dict:
    """yfiance 사용하여 특정 날짜의 주식의 가격 정보를 조회합니다."""

    if date and not is_valid_date(date):
        raise ToolException(f"잘못된 날짜 형식입니다: {date}")
    
    try:
        stock = yf.Ticker(symbol)
        # 특정 날짜의 주식 가격 정보 조회 
        if date:
            start = datetime.strptime(date, "%Y-%m-%d")
            end = start + timedelta(days=1)
            price = stock.history(start=start, end=end)

            # 가격 정보가 없으면 해날 날짜로부터 과거 5일간의 주식 가격 정보 조회
            if price.empty:
                end = start - timedelta(days=5)
                price = stock.history(start=end, end=start)

        # 특정 날짜가 없으면 최근 5일간의 주식 가격 정보 조회
        else:
            price = stock.history(period="5d")
            
        # 데이터프레임을 딕셔너리로 변환하여 반환 (가장 최근 날짜 데이터만 반환)
        df = price.reset_index()
        df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
        return df.to_dict(orient='records')[-1]
    
    except Exception as e:
        raise ToolException(str(e))
    

def is_valid_date(date_str: str) -> bool:
    try:
        datetime.strptime(date_str, '%Y-%m-%d')
        return True
    except ValueError:
        return False
    

# 함수 실행
result = get_stock_price("AAPL")

# 결과 출력
print(result)

{'Date': '2025-07-01', 'Open': 206.6699981689453, 'High': 210.19000244140625, 'Low': 206.13999938964844, 'Close': 207.82000732421875, 'Volume': 78673300, 'Dividends': 0.0, 'Stock Splits': 0.0}


In [21]:
# 함수 실행 (날짜 지정)
result = get_stock_price("AAPL", "2025-01-01")
print(result)

$AAPL: possibly delisted; no price data found  (1d 2025-01-01 00:00:00 -> 2025-01-02 00:00:00)


{'Date': '2024-12-31', 'Open': 251.83251072711272, 'High': 252.67048563095815, 'Low': 248.82974448856297, 'Close': 249.81736755371094, 'Volume': 39480700, 'Dividends': 0.0, 'Stock Splits': 0.0}


`(3) StructuredTool 도구 변환` 

In [22]:
from langchain_core.tools import StructuredTool

# StructuredTool로 도구 생성
stock_tool = StructuredTool.from_function(
    func=get_stock_price,
    name="stock_price_basic",
    description="yfinance를 사용하여 주식 가격 정보를 조회하는 도구입니다.",
)

# 도구 실행 (정상)
result = stock_tool.invoke({"symbol": "AAPL"})
print(result)

{'Date': '2025-07-01', 'Open': 206.6699981689453, 'High': 210.19000244140625, 'Low': 206.13999938964844, 'Close': 207.82000732421875, 'Volume': 78673300, 'Dividends': 0.0, 'Stock Splits': 0.0}


In [23]:
stock_tool.model_dump()

{'name': 'stock_price_basic',
 'description': 'yfinance를 사용하여 주식 가격 정보를 조회하는 도구입니다.',
 'args_schema': langchain_core.utils.pydantic.stock_price_basic,
 'return_direct': False,
 'verbose': False,
 'tags': None,
 'metadata': None,
 'handle_tool_error': False,
 'handle_validation_error': False,
 'response_format': 'content',
 'func': <function __main__.get_stock_price(symbol: str, date: Optional[str] = None) -> Dict>,
 'coroutine': None}

`(4) LLM 사용하여 도구 사용` 

In [24]:
from langchain_openai import ChatOpenAI

# OpenAI GPT-4o-mini 모델 사용
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# 도구 바인딩
llm_with_tool = llm.bind_tools([search_web, stock_tool])

# 도구 호출
result = llm_with_tool.invoke("애플의 주식 가격을 알려줘")
pprint(result.tool_calls)
print("-"*100)

# ToolCall을 도구에 전달하여 결과 확인
tool_call = result.tool_calls[0]
if tool_call['name'] == "stock_price_basic":
    print("도구 호출 이름:", tool_call['name'])
    result = stock_tool.invoke(tool_call)
elif tool_call['name'] == "tavily_search_results_json":
    print("도구 호출 이름:", tool_call['name'])
    result = search_web.invoke(tool_call)
else:
    raise ValueError(f"알 수 없는 도구 호출: {tool_call['name']}")

print(result)

[{'args': {'symbol': 'AAPL'},
  'id': 'call_ynhQWBz0UsHmKAFkBSRuFtKY',
  'name': 'stock_price_basic',
  'type': 'tool_call'}]
----------------------------------------------------------------------------------------------------
도구 호출 이름: stock_price_basic
content='{"Date": "2025-07-01", "Open": 206.6699981689453, "High": 210.19000244140625, "Low": 206.13999938964844, "Close": 207.82000732421875, "Volume": 78673300, "Dividends": 0.0, "Stock Splits": 0.0}' name='stock_price_basic' tool_call_id='call_ynhQWBz0UsHmKAFkBSRuFtKY'


In [25]:
# 도구 호출 (날짜 지정)
result = llm_with_tool.invoke("애플의 2025년 1월 3일 주식 가격을 알려줘")
pprint(result.tool_calls)

# ToolCall을 도구에 전달하여 결과 확인
for tool_call in result.tool_calls:
    print("도구 호출 이름:", tool_call['name'])
    if tool_call['name'] == "stock_price_basic":
        result = stock_tool.invoke(tool_call)
    elif tool_call['name'] == "tavily_search_results_json":
        result = search_web.invoke(tool_call)
    else:
        raise ValueError(f"알 수 없는 도구 호출: {tool_call['name']}")
    
    print(result)
    print("-" * 100)

[{'args': {'date': '2025-01-03', 'symbol': 'AAPL'},
  'id': 'call_mjeQjLydCW5Gek2ODHQrO6wp',
  'name': 'stock_price_basic',
  'type': 'tool_call'}]
도구 호출 이름: stock_price_basic
content='{"Date": "2025-01-03", "Open": 242.7743682861328, "High": 243.5923870862029, "Low": 241.3079045417172, "Close": 242.7743682861328, "Volume": 40244100, "Dividends": 0.0, "Stock Splits": 0.0}' name='stock_price_basic' tool_call_id='call_mjeQjLydCW5Gek2ODHQrO6wp'
----------------------------------------------------------------------------------------------------


In [26]:
from langgraph.prebuilt import create_react_agent

# 도구 실행 에인전트 생성 
stock_agent = create_react_agent(llm, tools=[search_web, stock_tool])

# 도구 실행 에이전트 사용
result = stock_agent.invoke(
    {"messages": [("human", "애플의 2025년 1월 3일 주식 가격을 알려줘")]}
)

pprint(result['messages'])

[HumanMessage(content='애플의 2025년 1월 3일 주식 가격을 알려줘', additional_kwargs={}, response_metadata={}, id='693b15c9-2997-485c-933a-926dc78341cc'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_QLc0CUNU7Ob3K4OKO3EChVl4', 'function': {'arguments': '{"symbol":"AAPL","date":"2025-01-03"}', 'name': 'stock_price_basic'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 140, 'total_tokens': 165, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_6f2eabb9a5', 'id': 'chatcmpl-BohC7xuG2gXj5eR6rB5cmMrE07EtE', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--dcc59fca-c9c6-41a8-ab5a-f7a2734ffc4b-0', tool_calls=[{'name': 'stock_price_basic', 'args': {'symbol': '

In [27]:
# 도구 실행 에인전트 생성 (return_direct=True)

stock_tool.return_direct = True
search_web.return_direct = True
stock_agent = create_react_agent(llm, tools=[search_web, stock_tool])

# 도구 실행 에이전트 사용
result = stock_agent.invoke(
    {"messages": [("human", "애플의 2025년 1월 3일 주식 가격을 알려줘")]}
)

pprint(result['messages'])

[HumanMessage(content='애플의 2025년 1월 3일 주식 가격을 알려줘', additional_kwargs={}, response_metadata={}, id='f638df10-f474-473b-af73-8e625d99f6d6'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_zRaxjrtjNwoDJYHyWLtTFwhS', 'function': {'arguments': '{"symbol":"AAPL","date":"2025-01-03"}', 'name': 'stock_price_basic'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 140, 'total_tokens': 165, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_6f2eabb9a5', 'id': 'chatcmpl-BohCv1ogEAwZv4cyJjMvKtfEuEKiB', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--bf5ade9d-250a-46dd-a837-7b69c4fd226d-0', tool_calls=[{'name': 'stock_price_basic', 'args': {'symbol': '

In [28]:
# 도구 실행 에이전트 사용
result = stock_agent.invoke(
    {"messages": [("human", "애플의 최신 뉴스를 알려줘")]}
)

pprint(result['messages'])

[HumanMessage(content='애플의 최신 뉴스를 알려줘', additional_kwargs={}, response_metadata={}, id='9a2eefaa-3e4d-4f66-906f-df84ae0b967f'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ZFJdeFXLp1peszQX9hqQt1Pb', 'function': {'arguments': '{"query":"Apple latest news"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 129, 'total_tokens': 149, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_6f2eabb9a5', 'id': 'chatcmpl-BohDKdsthLfyH5KMC6rjrmG9JurvR', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--773b0962-6c19-4aa4-b0da-658bec22f8a1-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'App

---

### 2. **Naver 개발자 API** 를 도구로 사용

- 네이버 개발자 API(https://developers.naver.com/)에서 인증 권한 취득 (회원 가입 및 애플리케이션 등록 필요)
- 환경변수(.env)를 등록합니다. (**NAVER_CLIENT_ID**, **NAVER_CLIENT_SECRET**)
- 아래 정의된 네이버 뉴스 검색 도구(naver_news_search)를 사용하여 다음 과정을 수행합니다. 

In [ ]:
# 환경 변수 로드
from dotenv import load_dotenv
load_dotenv()

`(1) 도구 정의`

In [29]:
import requests, os
from langchain_core.tools import tool
from typing import Dict

@tool
def naver_news_search(
    query: str,
    ) -> Dict[Dict, int]:
    """네이버 검색 API를 사용하여 뉴스 검색 결과를 조회합니다."""

    url = "https://openapi.naver.com/v1/search/news.json"
    headers = {
        "X-Naver-Client-Id": os.getenv("NAVER_CLIENT_ID"),
        "X-Naver-Client-Secret": os.getenv("NAVER_CLIENT_SECRET")
    }
    params = {"query": query}

    response = requests.get(url, headers=headers, params=params)

    return {
        "data": response.json(),
        "status_code": int(response.status_code)
    }  #type: ignore

`(2) 도구 실행`

In [30]:
from langchain_core.runnables import chain, RunnableLambda
from langchain_openai import ChatOpenAI

# 도구 맵 생성 (도구 이름을 키로 사용)
tools = [stock_tool, naver_news_search]  
tool_map = {tool.name: tool for tool in tools}

# 도구 라우터 (도구 이름에 따라 실행할 도구를 선택)
@chain
def tool_router(tool_call):
    return tool_map[tool_call["name"]]

# 체인 구성
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

llm_with_tools = llm.bind_tools(tools)

tool_chain = (
    llm_with_tools    # 어떤 도구를 사용할지 결정 (LLM 모델이 도구 호출을 처리)
    | RunnableLambda(lambda x: x.tool_calls)  # 도구 호출을 추출
    | tool_router.map()   # 도구 호출 라우팅
)
# 도구 실행
response = tool_chain.invoke("애플의 최근 주가는 어떻게 되나요? 최근 주가 분석에 대한 관련 기사를 찾아주세요.")

In [31]:
# 결과 출력 (도구 호출 결과)
for msg in response:
    pprint(msg)

ToolMessage(content='{"Date": "2025-07-01", "Open": 206.6699981689453, "High": 210.19000244140625, "Low": 206.13999938964844, "Close": 207.82000732421875, "Volume": 78673300, "Dividends": 0.0, "Stock Splits": 0.0}', name='stock_price_basic', tool_call_id='call_CZR42PioOBjJy7LQ1paJxKAL')
ToolMessage(content='{"data": {"lastBuildDate": "Wed, 02 Jul 2025 11:15:46 +0900", "total": 213957, "start": 1, "display": 10, "items": [{"title": "\'머스크-트럼프 갈등\' 심화에 테슬라 <b>주가</b> 5.3% 하락", "originallink": "http://www.newsfc.co.kr/news/articleView.html?idxno=72238", "link": "http://www.newsfc.co.kr/news/articleView.html?idxno=72238", "description": "테슬라 <b>주가</b>가 큰 폭으로 내렸다. 연합뉴스에 따르면 1일(현지시간) 뉴욕증시에서 테슬라 <b>주가</b>는 전날보다... 반면 <b>애플</b>(1.29%↑)과 아마존(0.49%↑)은 상승했다. 이날 트럼프 대통령은 상호관세 유예 기간 연장에 대해... ", "pubDate": "Wed, 02 Jul 2025 11:08:00 +0900"}, {"title": "코스피 3080대 출발… 뉴욕증시 혼조 마감", "originallink": "https://www.newsclaim.co.kr/news/articleView.html?idxno=3046511", "link": "https://www.newsclaim.co.kr/ne

`(3) 체인 실행`

In [32]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import chain
from langchain_core.messages import ToolMessage
from langchain_openai import ChatOpenAI
from datetime import datetime

# LLM 모델 인스턴스를 생성
llm = ChatOpenAI(model="gpt-4.1-mini")

# 두 도구를 LLM에 바인딩
llm_with_tools = llm.bind_tools(tools=[stock_tool, naver_news_search])

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 정의
prompt = ChatPromptTemplate([
    ("system", f"당신은 도움이 되는 AI 어시스턴트입니다. 오늘 날짜는 {today}입니다. 답변 생성의 근거 또는 출처를 명시하세요."),
    ("human", "{user_input}"),  
    ("placeholder", "{messages}"),
])

# LLM 체인 생성
llm_chain = prompt | llm_with_tools

@chain
def news_analysis_chain(user_input: str):

    # 도구 체인 실행 (LLM -> StockPrice/NaverNewsSearch)
    tool_msgs = tool_chain.invoke(user_input)

    print(f"Tool Messages: {tool_msgs}")

    # 도구 결과를 포맷팅
    formatted_messages = []
    for msg in tool_msgs:
        if isinstance(msg, ToolMessage) and len(msg.content) > 0:
            formatted_messages.append({"role": "assistant", "content": msg.content}) 

    print(f"Formatted Messages: {formatted_messages}")

    # 최종 응답 생성
    final_response = llm_chain.invoke({"user_input": user_input, "messages": formatted_messages})

    print(f"Final Response: {final_response}")

    return final_response

In [33]:
# 체인 실행
response = news_analysis_chain.invoke("애플의 최근 주가는 어떻게 되나요? 최근 주가 분석에 대한 관련 기사를 찾아주세요.")

Tool Messages: [ToolMessage(content='{"Date": "2025-07-01", "Open": 206.6699981689453, "High": 210.19000244140625, "Low": 206.13999938964844, "Close": 207.82000732421875, "Volume": 78673300, "Dividends": 0.0, "Stock Splits": 0.0}', name='stock_price_basic', tool_call_id='call_Z98Ppvlx622rkQCXNSuS9ZDt'), ToolMessage(content='{"data": {"lastBuildDate": "Wed, 02 Jul 2025 11:16:45 +0900", "total": 213957, "start": 1, "display": 10, "items": [{"title": "\'머스크-트럼프 갈등\' 심화에 테슬라 <b>주가</b> 5.3% 하락", "originallink": "http://www.newsfc.co.kr/news/articleView.html?idxno=72238", "link": "http://www.newsfc.co.kr/news/articleView.html?idxno=72238", "description": "테슬라 <b>주가</b>가 큰 폭으로 내렸다. 연합뉴스에 따르면 1일(현지시간) 뉴욕증시에서 테슬라 <b>주가</b>는 전날보다... 반면 <b>애플</b>(1.29%↑)과 아마존(0.49%↑)은 상승했다. 이날 트럼프 대통령은 상호관세 유예 기간 연장에 대해... ", "pubDate": "Wed, 02 Jul 2025 11:08:00 +0900"}, {"title": "코스피 3080대 출발… 뉴욕증시 혼조 마감", "originallink": "https://www.newsclaim.co.kr/news/articleView.html?idxno=3046511", "link": "https://www.n

In [34]:
# 최종 응답 출력 
print(response.content)

애플의 최근 주가는 2025년 7월 1일 기준으로 약 207.82달러로, 하루 동안 1.2% 상승한 상태입니다.

최근 뉴스 분석을 보면, 애플은 인도에 새로운 아이폰 공장을 설립하고 폭스콘과 협력 관계를 확장하면서 주가 상승을 기록했습니다. 또한 AI 관련 기술 투자와 '시리' 성능 개선 등도 긍정적인 영향을 준 것으로 보입니다. 최근 트럼프와 머스크 간의 갈등 속에서도 애플은 비교적 안정적이며 상승하는 모습을 보이고 있습니다.

상세한 관련 기사 링크도 참고하시면 도움이 될 것입니다:
- [애플 주가 상승 배경과 인도 공장 설립 소식](https://www.newsclaim.co.kr/news/articleView.html?idxno=3046511)
- [AI 기능 강화 및 '시리' 개선 기대감으로 주가 상승](https://news.dealsitetv.com/articles/156208)

원하시면 더 구체적인 뉴스 내용을 제공해 드릴 수 있습니다.


In [43]:
from langgraph.prebuilt import create_react_agent

stock_tool.return_direct = False
naver_news_search.return_direct = False

# 도구 실행 에인전트 생성
tools=[stock_tool, naver_news_search]
langgraph_agent_executor = create_react_agent(llm, tools, prompt="답변 생성에 사용한 뉴스의 근거 또는 출처를 명시하세요.")

# 에이전트 실행
messages = langgraph_agent_executor.invoke(
    {"messages": [("human", "애플의 최근 주가는 어떻게 되나요? 최근 주가 분석에 대한 관련 기사를 찾아주세요.")]}
)

# 에이전트 실행 결과 출력
pprint(messages)

{'messages': [HumanMessage(content='애플의 최근 주가는 어떻게 되나요? 최근 주가 분석에 대한 관련 기사를 찾아주세요.', additional_kwargs={}, response_metadata={}, id='2e503063-48e9-4913-8438-6db6839e3d12'),
              AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_7HVXUaNNjCJRgLzBohLztAdW', 'function': {'arguments': '{"symbol": "AAPL"}', 'name': 'stock_price_basic'}, 'type': 'function'}, {'id': 'call_pkrC5UnRw9SHIoASjPNSe4hW', 'function': {'arguments': '{"query": "애플 최근 주가"}', 'name': 'naver_news_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 135, 'total_tokens': 187, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_6f2eabb9a5', 'id': 'chatcmpl-BohODUSYiL0onZTRsV0OvRPpYnutQ', 'service_tier': 'def

In [44]:
print(messages['messages'][-1].content)

애플(AAPL)의 최근 주가는 2025년 7월 1일 기준으로 종가 207.82달러입니다.

관련 뉴스로는 최근 애플과 관련된 다양한 기사들이 있습니다. 일부 기사 제목과 링크를 안내해 드리겠습니다:

1. "메스킹 vs 트럼프 청문 계획…테슬라 시총 1조달러 법규" - [폴리뉴스 기사](https://www.polinews.co.kr/news/articleView.html?idxno=699536)
2. "일론 머스크 미국에서 추진?" - [이데일리 기사](https://n.news.naver.com/mnews/article/018/0006054295?sid=101)
3. "[굿모닝! 글로벌 뉴스] 상호간 우려 및 금리인하 가능성 등 불확실성" - [뉴스퀘스트 기사](https://www.newsquest.co.kr/news/articleView.html?idxno=247678)

더 자세한 뉴스 내용이나 추가 정보를 원하시면 말씀해 주세요.
